# Завдання
https://www.hackerrank.com/challenges/15-days-of-learning-sql/problem

# Запит `mysql 5` яким отримано конкатеноване групування ункікальних хакерів та кількості відправок в день

In [28]:
prepare_query = '''select 
     h1
    ,group_concat(distinct concat(s1,':',c1) order by s1 asc)
from     (select 
         hacker_id  as h1
        ,submission_date as s1
        ,count(submission_id) as c1
    from submissions
    -- where hacker_id in (39091,66648)
    group by 
         hacker_id
        ,submission_date
    order by 1,2 ) as q1
group by h1
'''
print(prepare_query)

select 
     h1
    ,group_concat(distinct concat(s1,':',c1) order by s1 asc)
from     (select 
         hacker_id  as h1
        ,submission_date as s1
        ,count(submission_id) as c1
    from submissions
    -- where hacker_id in (39091,66648)
    group by 
         hacker_id
        ,submission_date
    order by 1,2 ) as q1
group by h1



In [1]:
import pandas as pd
from pandasql import sqldf
import re

In [2]:
sql6 = pd.read_csv('edu - sql - 15 days of learning sql - sql6 - юзер - кількість відправок по дням.csv')

In [3]:
sql6

hacker_id submission_by_date
0        11587       2016-03-01:1
1        22467       2016-03-01:1
2        26170       2016-03-01:1
3        42225       2016-03-01:1
4        63800       2016-03-01:1
..         ...                ...
469      93433       2016-03-14:1
470       8871       2016-03-15:1
471      12856       2016-03-15:1
472      26023       2016-03-15:1
473      96104       2016-03-15:1

[474 rows x 2 columns]

# Розбивання агрегації (group_concat) до таблиці з рядками

In [7]:
submissions_table = []
for idx in range(len(sql6)):
    submission_by_date_lst = sql6.submission_by_date[idx].split(',')
    hacker_id = sql6.hacker_id[idx]
    
    for each in submission_by_date_lst:
        data = each.split(':')
        day = data[0]
        submissions = data[1]
        row = hacker_id,day,submissions
        submissions_table.append(row)
        #print(hacker_id,day,submissions)
        
    

## Формування назв колонок

In [8]:
submissions = pd.DataFrame(submissions_table)
submissions.columns = 'hacker_id,submissions_date,submissions'.split(',')

In [9]:
submissions['submissions_date'] = pd.to_datetime(submissions['submissions_date'])
submissions

hacker_id submissions_date submissions
0         11587       2016-03-01           1
1         22467       2016-03-01           1
2         26170       2016-03-01           1
3         42225       2016-03-01           1
4         63800       2016-03-01           1
...         ...              ...         ...
1880      93433       2016-03-14           1
1881       8871       2016-03-15           1
1882      12856       2016-03-15           1
1883      26023       2016-03-15           1
1884      96104       2016-03-15           1

[1885 rows x 3 columns]

In [14]:
submissions.submissions_date = pd.to_datetime(submissions.submissions_date)

# Кількість відправок по заданому дню

In [10]:
custom_value = '2016-03-02'

q = f'''
select *
from submissions a
where 
    date(submissions_date) = '{custom_value}'
    and submissions>1
order by 3 desc,1 desc 
'''
sqldf(q)

hacker_id            submissions_date submissions
0       98981  2016-03-02 00:00:00.000000           3
1       39091  2016-03-02 00:00:00.000000           3
2       99789  2016-03-02 00:00:00.000000           2
3       89781  2016-03-02 00:00:00.000000           2
4       87230  2016-03-02 00:00:00.000000           2
5       78408  2016-03-02 00:00:00.000000           2
6       73967  2016-03-02 00:00:00.000000           2
7       69030  2016-03-02 00:00:00.000000           2
8       66648  2016-03-02 00:00:00.000000           2
9       60516  2016-03-02 00:00:00.000000           2
10      21973  2016-03-02 00:00:00.000000           2

In [11]:

submissions[submissions['hacker_id']==59761] 

hacker_id submissions_date submissions
1057      59761       2016-03-02           1
1058      59761       2016-03-03           2
1059      59761       2016-03-06           1
1060      59761       2016-03-09           1
1061      59761       2016-03-11           1

# зведення по дням

In [12]:
q = '''
select
     date(a.submissions_date) as submissions_date
    ,group_concat(distinct a.hacker_id) as total
    ,count(distinct a.hacker_id) as dcnt
from submissions a
group by 
    1
'''

#'81314' in sqldf(q)['total'].iloc[0]
hackers_of_the_day = sqldf(q).values.tolist()

hackers_of_the_day_python = [  
    (int(i[0][-2:])
   ,i[1].split(',')) 
    for i in hackers_of_the_day]

#hackers_of_the_day_python
#hackers_of_the_day['submissions_date'] = hackers_of_the_day.submissions_date.dt.day

# -Обробка максимальних відправок вне залежності від кросс-джоін звязку

In [13]:
q = '''
with max_stat_by_day as
(select 
     submissions_date
    ,max(submissions) as max_submissions
from submissions
group by submissions_date
)


,only_max_hackers as 
(select 
     a.*
    ,min(b.hacker_id)  as min_hacker_id_with_max_submissions
    ,group_concat(distinct b.hacker_id) as only_max
from max_stat_by_day a
left join submissions b
    on a.submissions_date = b.submissions_date
    and a.max_submissions = b.submissions 
group by 
    a.submissions_date
)

select 
     d.*
    ,group_concat(distinct e.hacker_id) as total
    ,count(distinct e.hacker_id) as dcnt
from submissions e 
left join only_max_hackers d
    on date(d.submissions_date) = date(e.submissions_date) 
group by d.submissions_date
'''

#IN
#MAX
#
data_struct  = sqldf(q)#['dcnt'].sum() 
data_struct

submissions_date max_submissions  \
0   2016-03-01 00:00:00.000000               3   
1   2016-03-02 00:00:00.000000               3   
2   2016-03-03 00:00:00.000000               2   
3   2016-03-04 00:00:00.000000               2   
4   2016-03-05 00:00:00.000000               2   
5   2016-03-06 00:00:00.000000               4   
6   2016-03-07 00:00:00.000000               4   
7   2016-03-08 00:00:00.000000               2   
8   2016-03-09 00:00:00.000000               3   
9   2016-03-10 00:00:00.000000               2   
10  2016-03-11 00:00:00.000000               2   
11  2016-03-12 00:00:00.000000               3   
12  2016-03-13 00:00:00.000000               2   
13  2016-03-14 00:00:00.000000               3   
14  2016-03-15 00:00:00.000000               2   

    min_hacker_id_with_max_submissions  \
0                                81314   
1                                39091   
2                                18105   
3                                  533   
4                                 7891   
5                                84307   
6                                80682   
7                                10985   
8                                31221   
9                                43192   
10                                3178   
11                               54967   
12                               30061   
13                               32353   
14                               27789   

                                             only_max  \
0                                               81314   
1                                         39091,98981   
2                       91573,59761,24725,80682,18105   
3                   77217,30845,72783,533,78408,14737   
4   31822,40093,11753,68071,40816,7891,90612,57572...   
5                                               84307   
6                                               80682   
7                       10985,18105,50503,44690,54180   
8                                   95648,31221,62078   
9                             66648,43192,75908,51572   
10  92239,98727,60516,44690,39434,91095,13901,4170...   
11                                              54967   
12                            30061,81242,53940,89457   
13                                        74861,32353   
14                                  72237,80648,27789   

                                                total  dcnt  
0   11587,22467,26170,42225,63800,67815,81338,8444...   112  
1   83913,650,3162,4413,5478,5509,5990,7460,7865,7...   144  
2   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   122  
3   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   136  
4   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   144  
5   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   140  
6   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   101  
7   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   147  
8   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   154  
9   650,3162,4413,5478,5509,5990,7460,7865,7897,79...   108  
10  650,3162,4413,5478,5509,5990,7460,7865,7897,79...   117  
11  650,3162,4413,5478,5509,5990,7460,7865,7897,79...   107  
12  650,3162,4413,5478,5509,5990,7460,7865,7897,79...    90  
13  650,3162,4413,5478,5509,5990,7460,7865,7897,79...   146  
14  650,3162,4413,5478,5509,5990,7460,7865,7897,79...   117

In [14]:
'81314' in data_struct['total'].iloc[0]

True

In [15]:
def factorial(n):
    if n>=2:
        res = 0
        for i in range(n-1,1,-1):
            n = n*i
            res = n
            
    return res

factorial(4)

24

In [16]:
def factorial(n):
    if n==1:
        return 1
    else:
        return n*factorial(n-1)
    
factorial(4)

24

In [17]:
#for i in range(0,2):
for i in range(len(data_struct)):
    top_hackers = data_struct['total'][i].split(',')
    if i!=0:
        
        prev_day_all_hackers = data_struct['total'][i-1].split(',')
        prev_day_validation = [top for top in top_hackers 
                               if top in prev_day_all_hackers]
        
        if i<len(data_struct)-1:
            next_day_all_hackers = data_struct['total'][i+1].split(',')
            next_day_filter =  len([i for i in prev_day_validation 
                                    if i in next_day_all_hackers])
            
            print(i,next_day_filter)
        
#idx+1
#

1 51
2 55
3 53
4 55
5 35
6 36
7 37
8 63
9 37
10 40
11 39
12 48
13 47


# +Функція для кросс-джоіну, послідовних перетинів та перезапису

In [18]:
lst = [ #note допомогло. це крок вперед
 (1, [2,3,4,5,6] )
,(2, [2,3,4 ]    )
,(3, [4,5,6,2]   )
,(4, [2,3,4]     )
,(5, [3,2]       ) ]

def triangle(lst):
    c=0
    day_state = []
    for i in lst:
        for j in lst:
            if i!=j and i[0]==1 and j[0]>1: #note джоінемо тільки до першого дня only if first day
                left        = list(intesection) if c!=0 else i[1] #note тернатрий оператор допомагає перезаписати список
                right       = j[1]
                subset1     = set(left)
                subset2     = set(right)
                intesection = subset1&subset2 #пертини першого дня з перезаписами множини left
                c+=1
                #print(left,intesection)
                row = ( j[0]-1
                       ,len(left)
                       ,left) #note останній день буде пропущено, додано нижче
                
                day_state.append(row)
                
    #last = [(i[0]+1,list(intesection)) for i in day_state][-1]
    last = ( day_state[-1][0]+1
            ,len(intesection)
            ,list(intesection)
           ) #note нюанс додаємо останнє значення перетину
    day_state.append(last) 
    return day_state

triangle(lst)

[(1, 5, [2, 3, 4, 5, 6]),
 (2, 3, [2, 3, 4]),
 (3, 2, [2, 4]),
 (4, 2, [2, 4]),
 (5, 1, [2])]

In [19]:
%%time
data_processing = triangle(hackers_of_the_day_python)

#[i[1] for i in data_processing if '11587' in str(i)]

Wall time: 0 ns


In [25]:
data_processing[0][0:2]

(1, 112)

# Додавання дати 
(бо структура вимагає індекс дня в рамках місяця) та популяція рядків

In [25]:
data_population = []
for i in data_processing:
    values = i[-1]
    for value in values:
        row = f'2016-03-{str(i[0]).zfill(2)}',i[1],value
        data_population.append(row)
        
        
data_population_df = pd.DataFrame(data_population)
#data_population_df
data_population_df.columns = 'submissions_date,hackers_amount,hacker_id'.split(',')
data_population_df['submissions_date'] = pd.to_datetime(data_population_df['submissions_date'])
data_population_df

submissions_date  hackers_amount hacker_id
0         2016-03-01             112     11587
1         2016-03-01             112     22467
2         2016-03-01             112     26170
3         2016-03-01             112     42225
4         2016-03-01             112     63800
..               ...             ...       ...
679       2016-03-15              35      8150
680       2016-03-15              35     24095
681       2016-03-15              35     22917
682       2016-03-15              35      5990
683       2016-03-15              35     21372

[684 rows x 3 columns]

In [29]:
#data_population_df[data_population_df[0].str.contains('11587')]

In [28]:
#data_population_df[data_population_df['submissions_date'].astype(str).str.contains('11587')]

# Res

In [ ]:
'''
-- зайве
,case when max_submissions>1 then max(b.hacker_id) 
          when max_submissions=1 then min(b.hacker_id) 
          end as c -- note не очікував розрахунку тільки однієї агрегації
    ,count(distinct (case when max_submissions then b.hacker_id end)) as dcnt_only_max_submissions

In [79]:
q = '''
with tmp as 
(select 
     a.*
    ,b.submissions
    -- ,count(a.hacker_id) as hackers_cnt --
from data_population_df a 
left join submissions b
    on a.hacker_id = b.hacker_id
    and date(a.submissions_date) = date(b.submissions_date)
)

,max_submissions_value as (
select 
      submissions_date
     ,count(distinct hacker_id) as dcnt -- note кількість унікальних хакерів в ден, які займалися ланцюгом кожен день
     ,max(submissions) as max_submissions
from tmp
group by submissions_date
)

select 
     date(a.submissions_date)
    -- ,a.max_submissions
    ,a.dcnt
    ,c.min_hacker_id_with_max_submissions -- note мінімальне ід хакера при зєданні з максимальною кількістю відправок
from max_submissions_value a
left join submissions b
    on a.submissions_date = b.submissions_date
    and a.max_submissions = b.submissions
left join data_struct c
    on c.submissions_date = a.submissions_date
group by 
    1
'''

res = sqldf(q).values.tolist()
res

sql_res_string = []
for i in res:
    row = 'select\t'+',\t'.join(["'"+k+"'" if type(k)!=int else str(k) for k in i])
    sql_res_string.append(row)
    
sql_res_string = '\tUNION ALL\n'.join(sql_res_string)
print(sql_res_string)

select	'2016-03-01',	112,	81314	UNION ALL
select	'2016-03-02',	59,	39091	UNION ALL
select	'2016-03-03',	51,	18105	UNION ALL
select	'2016-03-04',	49,	533	UNION ALL
select	'2016-03-05',	49,	7891	UNION ALL
select	'2016-03-06',	49,	84307	UNION ALL
select	'2016-03-07',	35,	80682	UNION ALL
select	'2016-03-08',	35,	10985	UNION ALL
select	'2016-03-09',	35,	31221	UNION ALL
select	'2016-03-10',	35,	43192	UNION ALL
select	'2016-03-11',	35,	3178	UNION ALL
select	'2016-03-12',	35,	54967	UNION ALL
select	'2016-03-13',	35,	30061	UNION ALL
select	'2016-03-14',	35,	32353	UNION ALL
select	'2016-03-15',	35,	27789


# Запит з даними - Для перевірки з результатом

In [63]:
print(f'''select 
      q.*
     ,hello_hacker.name
from (
-- тут треба дати аліас для хакерів
{sql_res_string}) as q
left join hackers hello_hacker
    on q.hi = hello_hacker.hacker_id
order by 1 asc-- не очікував сортування
''')

select 
      q.*
     ,hello_hacker.name
from (
select	'2016-03-01',	112,	81314	UNION ALL
select	'2016-03-02',	59,	39091	UNION ALL
select	'2016-03-03',	51,	18105	UNION ALL
select	'2016-03-04',	49,	533	UNION ALL
select	'2016-03-05',	49,	7891	UNION ALL
select	'2016-03-06',	49,	84307	UNION ALL
select	'2016-03-07',	35,	80682	UNION ALL
select	'2016-03-08',	35,	10985	UNION ALL
select	'2016-03-09',	35,	31221	UNION ALL
select	'2016-03-10',	35,	43192	UNION ALL
select	'2016-03-11',	35,	3178	UNION ALL
select	'2016-03-12',	35,	54967	UNION ALL
select	'2016-03-13',	35,	30061	UNION ALL
select	'2016-03-14',	35,	32353	UNION ALL
select	'2016-03-15',	35,	27789) as q
left join hackers hello_hacker
    on q.hi = hello_hacker.hacker_id
order by 1 asc-- не очікував сортування



In [ ]:
'''2016-03-01 112 81314 Denise 
2016-03-02 59 39091 Ruby 
2016-03-03 51 18105 Roy 
2016-03-04 49 533 Patrick 
2016-03-05 49 7891 Stephanie 
2016-03-06 49 84307 Evelyn 
2016-03-07 35 80682 Deborah 
2016-03-08 35 10985 Timothy 
2016-03-09 35 31221 Susan 
2016-03-10 35 43192 Bobby 
2016-03-11 35 3178 Melissa 
2016-03-12 35 54967 Kenneth 
2016-03-13 35 30061 Julia 
2016-03-14 35 32353 Rose 
2016-03-15 35 27789 Helen'''

In [448]:
#650

submissions[submissions['hacker_id']==18105]

hacker_id submissions_date submissions
1336      18105       2016-03-03           2
1337      18105       2016-03-07           1
1338      18105       2016-03-08           2
1339      18105       2016-03-12           1
1340      18105       2016-03-13           1

In [433]:
submissions[submissions['submissions_date']=='2016-03-03'].sort_values(by='hacker_id')

hacker_id submissions_date submissions
12          650       2016-03-03           1
777        2697       2016-03-03           1
27         3162       2016-03-03           1
42         4413       2016-03-03           1
57         5478       2016-03-03           1
...         ...              ...         ...
1322      90401       2016-03-03           1
788       91573       2016-03-03           2
1298      95647       2016-03-03           1
1320      95958       2016-03-03           1
1045      97336       2016-03-03           1

[122 rows x 3 columns]

# Incorrect

In [186]:
#if
#cross join
#reassign
#

for i in range(len(lst)):
    cur_val_arr = lst[i][-1]
    if i!=0:
        prev_val_arr = lst[i-1][-1]
        cur_filter = [cur for cur in cur_val_arr if cur in prev_val_arr]
        print(cur_filter)
    

#listcomp:
#if - filter
#if-else - ternary
#range(len) - cur & prev & next
#zip - x,y
#listcomp in listcomp
#sum, len

#no:
#increment
#assign


[2, 3, 4]
[4]
[4]
[3, 2]


In [385]:
test_cases = [
     (39091,  'valid.2016-03-01:1 and 2016-03-02:3')
    ,(90612,  'reject.idx4.first starts with 2016-03-04')
    ,(7891,'reject.idx4.')
    ,(40816,'reject.idx4.')
    ,(40093,'reject.idx4.')
    ,(81314,'зявляється лише в перший день з максимальним значенням')
    ,(83913,'')
    ,(91573,'')]


to_test = test_cases[-1][0]

submissions[submissions['hacker_id']==to_test]

hacker_id submissions_date submissions
787      91573       2016-03-01           1
788      91573       2016-03-03           2
789      91573       2016-03-05           1
790      91573       2016-03-11           1
791      91573       2016-03-15           1

In [30]:
q_only_15_days = '''
select 
     hacker_id,count(distinct submissions_date)
from submissions 
group by hacker_id
having count(distinct submissions_date)>1
order by count(distinct submissions_date) desc
'''

sqldf(q)

hacker_id  count(distinct submissions_date)
0        36594                                15
1        34369                                15
2        34234                                15
3        33863                                15
4        33323                                15
..         ...                               ...
370       5327                                 2
371       3178                                 2
372       2820                                 2
373       2764                                 2
374       1050                                 2

[375 rows x 2 columns]

In [86]:
q = '''
with shift_delta_calc as 
(select 
     hacker_id
    ,submissions_date
    ,lead(submissions_date) over(partition by hacker_id order by submissions_date asc) as shift_date
    ,julianday(lead(submissions_date) 
     over(partition by hacker_id order by submissions_date asc)) - julianday(submissions_date)
     as shift_delta
    ,submissions
from submissions 
order by 1 asc,2 asc)


,first_day as (
select * 
from submissions 
where date(submissions_date)='2016-03-01'
order by 3 desc

)


,only_one_day_step as
(select *
from shift_delta_calc
where 
    shift_delta=1
    and date(submissions_date)>'2016-03-01' 
    and hacker_id in (select hacker_id from first_day)
)

,first_day_exceptions as 
(select *
from first_day 
where hacker_id not in (select hacker_id from only_one_day_step)
)


,sample as (
select 
     hacker_id
    ,submissions_date
    ,submissions
from first_day
union all
select 
     hacker_id
    ,submissions_date
    ,submissions
from only_one_day_step)



,max_with_sequence as 
(select 
  submissions_date
 ,max(submissions) as max_submissions
from sample
group by submissions_date
)

,query_res as 
(select 
     a.*
    ,max(b.hacker_id)
    ,group_concat(distinct b.hacker_id)
from max_with_sequence a
left join sample b
    on a.submissions_date = b.submissions_date
    and a.max_submissions = b.submissions
group by a.submissions_date)

select *
from query_res
'''

#-1 day
#also 15tries

sqldf(q)

submissions_date max_submissions  max(b.hacker_id)  \
0   2016-03-01 00:00:00.000000               3             81314   
1   2016-03-02 00:00:00.000000               1             56707   
2   2016-03-03 00:00:00.000000               1             56707   
3   2016-03-04 00:00:00.000000               1             81314   
4   2016-03-05 00:00:00.000000               1             56707   
5   2016-03-06 00:00:00.000000               1             36594   
6   2016-03-07 00:00:00.000000               1             83371   
7   2016-03-08 00:00:00.000000               1             56707   
8   2016-03-09 00:00:00.000000               1             56707   
9   2016-03-10 00:00:00.000000               1             70115   
10  2016-03-11 00:00:00.000000               1             97530   
11  2016-03-12 00:00:00.000000               1             64790   
12  2016-03-13 00:00:00.000000               1             54408   
13  2016-03-14 00:00:00.000000               1             56707   

                   group_concat(distinct b.hacker_id)  
0                                               81314  
1   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
2   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
3   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
4   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
5   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
6   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
7   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
8   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
9   650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
10  650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
11  650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
12  650,3162,4413,5478,5509,5990,7460,7865,7897,79...  
13  650,3162,4413,5478,5509,5990,7460,7865,7897,79...

In [82]:
submissions[submissions['hacker_id']==650]

hacker_id submissions_date submissions
10        650       2016-03-01           1
11        650       2016-03-02           1
12        650       2016-03-03           1
13        650       2016-03-04           1
14        650       2016-03-05           1
15        650       2016-03-06           1
16        650       2016-03-07           1
17        650       2016-03-08           1
18        650       2016-03-09           1
19        650       2016-03-10           1
20        650       2016-03-11           1
21        650       2016-03-12           1
22        650       2016-03-13           1
23        650       2016-03-14           1
24        650       2016-03-15           1